In [1]:
import numpy as np
import re
import string
import unicodedata
from unicodedata import normalize
import sys
import indicnlp
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import array
from numpy import asarray
from numpy import zeros
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.utils import plot_model
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models.fasttext import FastText
import gensim
from gensim.models import Word2Vec

In [2]:
def load_doc(filename):
    file = open(filename , mode ='rt' , encoding = 'utf-8')
    text = file.read()
    file.close()
    return text

In [3]:
def to_pairs(text):
    lines = text.strip().split('\n') #1d list of strings
    eng_hin_pairs = [line.split('\t') for line in lines] #list
    eng_hin_pairs = np.delete(eng_hin_pairs , [0,1,2,3,4] , axis = 1)
#     eng_hin_pairs = np.delete(eng_hin_pairs , 0 , axis = 1)
    eng_hin_pairs = eng_hin_pairs.tolist()
    return eng_hin_pairs   

In [4]:
doc = load_doc('hindi-visual-genome-train.txt')
print(doc)

hin_eng_pairs = to_pairs(doc)

english_sentences = []
hindi_sentences = []
for i in range(len(hin_eng_pairs)):
    english_sentences.append(hin_eng_pairs[i][0])
    hindi_sentences.append(hin_eng_pairs[i][1])
len(hin_eng_pairs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



28930

In [5]:
hin_eng_pairs

[['it is an indoor scene', 'यह एक इनडोर दृश्य है'],
 ['Computer screens turned on', 'कंप्यूटर स्क्रीन चालू'],
 ['man has short hair', 'आदमी के छोटे बाल हैं'],
 ["photo album open on an adult's lap", 'एक वयस्क की गोद में फोटो एल्बम खुला'],
 ['there is a group of girls beside the black car',
  'काली कार के पास लड़कियों का एक समूह है'],
 ['Child in a stroller', 'एक घुमक्कड़ में बच्चा'],
 ['Tall metal lightpost', 'लंबा धातु प्रकाश पोस्ट'],
 ['wall is painted white', 'दीवार सफेद रंग की है'],
 ['there are several pictures on the wall', 'दीवार पर कई चित्र हैं'],
 ['woman facing the ocean', 'सागर का सामना कर रही महिला'],
 ['this is an office layout', 'यह एक कार्यालय लेआउट है'],
 ['four metallic chairs', 'चार धातु की कुर्सियाँ'],
 ['Clutter is on a table', 'क्लटर एक मेज पर है'],
 ['a white microwave oven', 'एक सफेद माइक्रोवेव ओवन'],
 ['White SUV driving through intersection',
  'चौराहे के माध्यम से सफेद एसयूवी ड्राइविंग'],
 ['Person crossing street with umbrella', 'छतरी के साथ सड़क पार करते व्य

In [6]:
len(hin_eng_pairs)

28930

In [7]:
print(type(hin_eng_pairs))

<class 'list'>


In [8]:
hindi_sentences

['यह एक इनडोर दृश्य है',
 'कंप्यूटर स्क्रीन चालू',
 'आदमी के छोटे बाल हैं',
 'एक वयस्क की गोद में फोटो एल्बम खुला',
 'काली कार के पास लड़कियों का एक समूह है',
 'एक घुमक्कड़ में बच्चा',
 'लंबा धातु प्रकाश पोस्ट',
 'दीवार सफेद रंग की है',
 'दीवार पर कई चित्र हैं',
 'सागर का सामना कर रही महिला',
 'यह एक कार्यालय लेआउट है',
 'चार धातु की कुर्सियाँ',
 'क्लटर एक मेज पर है',
 'एक सफेद माइक्रोवेव ओवन',
 'चौराहे के माध्यम से सफेद एसयूवी ड्राइविंग',
 'छतरी के साथ सड़क पार करते व्यक्ति',
 'इमारत पर झुकाव में आदमी ग्रे पैंट में',
 'भवन पर खिड़की',
 'कारों के बीच में खड़ा एक आदमी',
 'दीवार पर चित्रकला',
 'कुर्सी का पैर',
 'एक नाव की खिड़की',
 'कैबिनेट लकड़ी है',
 'एक आदमी एक किताब पढ़ रहा है',
 'पानी में तैरता हुआ व्यक्ति',
 'चमकती छाया के साथ दीपक',
 'प्लेट पर फल',
 'एक चीनी मिट्टी के बरतन सफ़ेद प्लेट',
 'काला खेल नियंत्रक',
 'कार की डिक्की में कुछ डालने वाला व्यक्ति।',
 'एक सड़क के साथ ग्रीन कार पार्क किया',
 'गली पर कार',
 'निर्माण कर्मचारी कवरलेस पहनते हैं',
 'पीले दरवाजे के साथ एक काली कार',
 '

In [9]:
def clean_english_data(lines):
    re_print = re.compile('[^%s]' %re.escape(string.printable))
    table = str.maketrans('' , '' , string.punctuation)
    clean_eng_lines = []
    for line in lines:
        line = normalize('NFD' , line).encode('ascii' , 'ignore')
        line = line.decode('UTF-8')
        line = line.split()
        line = [word.lower() for word in line]
        line = [word.translate(table) for word in line]
        line = [re_print.sub('' , w) for w in line]
        line = [word for word in line if word.isalpha()]
        line = ' '.join(line)
        clean_eng_lines.append(line)
    return clean_eng_lines    

In [10]:
clean_eng_lines = clean_english_data(english_sentences)

# print(english_sentences)
print(english_sentences[3])
print(clean_eng_lines[3])

photo album open on an adult's lap
photo album open on an adults lap


In [11]:
INDIC_NLP_LIB_HOME=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\FastText_large_dataset\anoopkunchukuttan-indic_nlp_library-eccde81"
INDIC_NLP_RESOURCES=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\FastText_large_dataset\indic_nlp_resources-master"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

In [12]:
def clean_text(line):
    text = line
    text = text.replace(u',' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u'(' ,'')
    text = text.replace(u')' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u':' ,'')
    text = text.replace(u"'" ,'')
    text = text.replace(u"'" ,'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text = re.sub('[a-zA-Z]' , '' , text)
    text = re.sub('[0-9+\-*/.%&!]' , '' , text)
    return text

In [13]:
def clean_hindi_data(lines):
    clean_hin_lines = []
    for line in lines:
        remove_nuktas = True
        factory = IndicNormalizerFactory()
        normalizer = factory.get_normalizer("hi" , remove_nuktas = True )
        line = clean_text(line)
        tokens = list()
        for t in indic_tokenize.trivial_tokenize(line):
            tokens.append(t)
        line = tokens
        line = [ word.lower() for word in line]
        line = [word for word in line if not re.search(r'\d', word)]
        line = ' '.join(line)
        clean_hin_lines.append(line)
    return clean_hin_lines  

In [14]:
clean_hindi_lines = clean_hindi_data(hindi_sentences)
print(hindi_sentences[133])
print(clean_hindi_lines[133])

तार से जुड़ा तार
तार से जुड़ा तार


In [15]:
len(clean_eng_lines)

28930

In [16]:
len(clean_hindi_lines)

28930

In [17]:
input_train , input_val, target_train, target_val = train_test_split(clean_eng_lines , clean_hindi_lines , test_size = 0.2, random_state = 42)

### Adding SOS and EOS ,  PREPARING INPUTS FOR ENCODER AND DECODER

In [18]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

In [19]:
#input sentences
input_sentences = input_train

#output sentences
for line in target_train:
    line = line + ' <eos>'
    output_sentences.append(line)

#output sentence input

for line in target_train:
    line = '<sos> ' + line
    output_sentences_inputs.append(line)


In [20]:
print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))
print("num samples output input:", len(output_sentences_inputs))

num samples input: 23144
num samples output: 23144
num samples output input: 23144


In [21]:
BATCH_SIZE = 64
EPOCHS = 10
LSTM_NODES =256
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

In [22]:
# for english vocabulary

In [23]:
eng_lang_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
eng_lang_tokenizer.fit_on_texts(clean_eng_lines)
input_integer_seq = eng_lang_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = eng_lang_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 5550
Length of longest sentence in input: 24


In [24]:
# for hindi vocabulary
hindi = []
for i in clean_hindi_lines:
    i = '<sos> ' + i + ' <eos>'
    hindi.append(i)
len(hindi)

hin_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
hin_lang_tokenizer.fit_on_texts(hindi)
output_integer_seq = hin_lang_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = hin_lang_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = hin_lang_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 7013
Length of longest sentence in the output: 33


In [25]:
# pad encoder inputs and decoder inputs
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
print("encoder_input_sequences[172]:", encoder_input_sequences[172])

decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])

decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')

encoder_input_sequences.shape: (23144, 24)
encoder_input_sequences[172]: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 34 17 41]
decoder_input_sequences.shape: (23144, 33)
decoder_input_sequences[172]: [ 2 85 88 61  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0]


### word embeddings

In [26]:
# english embeddings
english = [line.split() for line in clean_eng_lines]

In [27]:
# Defining values for parameters
embedding_size = 100
window_size = 5
min_word = 5
down_sampling = 1e-2

fast_text_model = FastText(english, 
                           vector_size = embedding_size,
                           window = window_size,
                           min_count = min_word,
                           sample = down_sampling,
                           workers = 4,
                           sg = 1,
                           epochs =100)

In [28]:
fast_text_model.save("FT_model")
#load saved gensim fastext model
fast_text_model = Word2Vec.load("FT_model")

In [29]:
fast_text_model.wv['book']

array([-0.31470472,  0.78434795, -0.7775383 ,  0.65888226, -1.2912425 ,
       -0.22280216,  0.10088294,  0.03334308,  0.70953435, -0.25155953,
        0.30113384, -0.42154253, -0.30148914,  0.31177464, -0.14246956,
        0.5855759 , -0.09018487,  0.68125343, -1.111528  , -0.57700694,
       -0.3002802 , -0.5517959 ,  0.08936477,  0.05015181, -0.4261154 ,
        0.11419763, -0.62581897,  0.07099982, -0.98030514,  0.04559356,
        0.24188405,  0.5209477 ,  0.58759826, -1.0248742 , -0.82122225,
        0.6983178 ,  1.1378686 , -0.36924523,  0.24938603, -0.68661094,
       -0.760347  ,  0.2995245 , -0.67338306, -0.2852774 ,  0.25930622,
        0.2881126 , -0.30679825,  0.4042898 ,  0.07425616, -0.5368603 ,
        0.32557237,  0.6185624 , -0.27499512,  0.03598544, -0.34126288,
       -0.8016766 , -0.35307708, -0.8344991 ,  0.11500187,  0.27773824,
       -0.778273  ,  0.5979548 ,  0.33765543, -0.3381162 ,  0.33250123,
        0.13158785, -0.33014658, -0.21995111, -0.6661435 , -0.40

In [30]:
fast_text_model.wv.get_vector('book')

array([-0.31470472,  0.78434795, -0.7775383 ,  0.65888226, -1.2912425 ,
       -0.22280216,  0.10088294,  0.03334308,  0.70953435, -0.25155953,
        0.30113384, -0.42154253, -0.30148914,  0.31177464, -0.14246956,
        0.5855759 , -0.09018487,  0.68125343, -1.111528  , -0.57700694,
       -0.3002802 , -0.5517959 ,  0.08936477,  0.05015181, -0.4261154 ,
        0.11419763, -0.62581897,  0.07099982, -0.98030514,  0.04559356,
        0.24188405,  0.5209477 ,  0.58759826, -1.0248742 , -0.82122225,
        0.6983178 ,  1.1378686 , -0.36924523,  0.24938603, -0.68661094,
       -0.760347  ,  0.2995245 , -0.67338306, -0.2852774 ,  0.25930622,
        0.2881126 , -0.30679825,  0.4042898 ,  0.07425616, -0.5368603 ,
        0.32557237,  0.6185624 , -0.27499512,  0.03598544, -0.34126288,
       -0.8016766 , -0.35307708, -0.8344991 ,  0.11500187,  0.27773824,
       -0.778273  ,  0.5979548 ,  0.33765543, -0.3381162 ,  0.33250123,
        0.13158785, -0.33014658, -0.21995111, -0.6661435 , -0.40

In [31]:
embeddings_dictionary = {}
for word in eng_lang_tokenizer.word_index:
    embeddings_dictionary[word] = fast_text_model.wv.get_vector(word)
    
print("the no of key-value pairs:"  , len(embeddings_dictionary))

the no of key-value pairs: 5550


In [32]:
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = zeros((num_words, EMBEDDING_SIZE))
for word, index in word2idx_inputs.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [33]:
# hindi word embeddings
model_h = gensim.models.Word2Vec.load('fasttext\hi-d100-m2-fasttext.model')

In [34]:
embeddings_dictionary_h = {}
for word in hin_lang_tokenizer.word_index:
    try:
        embeddings_dictionary_h[word] = model_h.wv.get_vector(word)
    except:
        pass
    
print("the no of key-value pairs:"  , len(embeddings_dictionary_h))
embeddings_dictionary_h

the no of key-value pairs: 7013


{'<OOV>': array([-0.13615343, -0.0276705 , -0.32373258, -0.33983436, -0.25074765,
         0.1136819 ,  0.06906117,  0.13111897,  0.37115008, -0.18939647,
         0.33253947, -0.09550375,  0.03981497, -0.00756057,  0.0919835 ,
         0.10029203,  0.00405759,  0.0080026 ,  0.01395009,  0.03374427,
         0.11002628, -0.32754344, -0.29076695, -0.15364802, -0.10591869,
         0.0816481 , -0.05164264,  0.18296564, -0.03330514, -0.3384538 ,
        -0.0645583 ,  0.3143947 , -0.09430255,  0.00978532,  0.1626066 ,
         0.20863284, -0.25920966, -0.24931912, -0.08786643,  0.08610689,
         0.13213517,  0.08543954,  0.18012294,  0.05626457,  0.23760124,
         0.38632652, -0.05079996,  0.03733306,  0.05593262, -0.08317049,
         0.07317774,  0.03252045,  0.22009286, -0.24432637,  0.01289156,
         0.09061732,  0.1443138 , -0.12644455,  0.22806767,  0.24784693,
         0.1422608 , -0.05146387, -0.04265721,  0.00522736,  0.2764139 ,
        -0.17195396,  0.15091114, -0.05463

In [35]:
num_words_h = min(MAX_NUM_WORDS, len(word2idx_outputs) + 1)
embedding_matrix_h = zeros((num_words_h, 100))
for word, index in hin_lang_tokenizer.word_index.items(): #input_lang_word2idx=1847
    embedding_vector = embeddings_dictionary_h.get(word)
    if embedding_vector is not None:
        embedding_matrix_h[index] = embedding_vector

# MODEL

In [36]:
## CREATING THE MODEL

In [37]:
decoder_targets_one_hot = np.zeros((len(input_sentences), max_out_len, num_words_output),dtype='float32')

In [38]:
decoder_targets_one_hot.shape
for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

## create the encoder and decoders

In [39]:
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [40]:
embedding_layer_h = Embedding(num_words_h, EMBEDDING_SIZE, weights=[embedding_matrix_h], input_length=max_out_len)

decoder_inputs_placeholder = Input(shape=(max_out_len,))
decoder_inputs_x = embedding_layer_h(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES , return_sequences = True , return_state = True)

decoder_outputs , _ , _ = decoder_lstm(decoder_inputs_x , initial_state = encoder_states)

In [41]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## compile the model:

In [42]:
model = Model([encoder_inputs_placeholder, decoder_inputs_placeholder] , decoder_outputs)

model.compile(optimizer = 'rmsprop' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [43]:
!pip install pydot
!pip install graphviz

In [44]:
# plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

In [45]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=10,
    validation_split=0.1,
)

Epoch 1/10
326/326 [==============================] - 203s 623ms/step - loss: 1.0330 - accuracy: 0.8572 - val_loss: 0.7886 - val_accuracy: 0.8801
Epoch 2/10
326/326 [==============================] - 197s 603ms/step - loss: 0.6828 - accuracy: 0.8925 - val_loss: 0.6233 - val_accuracy: 0.9001
Epoch 3/10
326/326 [==============================] - 197s 603ms/step - loss: 0.5399 - accuracy: 0.9116 - val_loss: 0.5193 - val_accuracy: 0.9162
Epoch 4/10
326/326 [==============================] - 198s 607ms/step - loss: 0.4506 - accuracy: 0.9248 - val_loss: 0.4602 - val_accuracy: 0.9252
Epoch 5/10
326/326 [==============================] - 197s 606ms/step - loss: 0.3919 - accuracy: 0.9335 - val_loss: 0.4210 - val_accuracy: 0.9303
Epoch 6/10
326/326 [==============================] - 198s 608ms/step - loss: 0.3505 - accuracy: 0.9399 - val_loss: 0.3987 - val_accuracy: 0.9342
Epoch 7/10
326/326 [==============================] - 197s 603ms/step - loss: 0.3188 - accuracy: 0.9447 - val_loss: 0.3831 -

In [46]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [47]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [48]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = embedding_layer_h(decoder_inputs_single)

In [49]:
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

In [50]:
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

In [51]:
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [52]:
# from keras.utils import plot_model
# plot_model(decoder_model, to_file='model_plot_dec.png', show_shapes=True, show_layer_names=True)

In [53]:
### predictions

In [54]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [55]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [56]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)

-
Input: sunshine on long grass
Response: लंबी घास पर


In [57]:
test_sentences = input_val
test_sentences_hindi = target_val

In [58]:
print(len(test_sentences) , len(test_sentences_hindi))

5786 5786


In [59]:
test_sentences.index('')

3575

In [60]:
test_sentences.pop(3575) , test_sentences_hindi.pop(3575)

('', 'समुद्र तट पर बाइक')

In [61]:
print(len(test_sentences) , len(test_sentences_hindi))

5785 5785


In [62]:
test_sentences.pop(1346)
test_sentences_hindi.pop(1346)


''

In [63]:
test_sentences.pop(1655)
test_sentences_hindi.pop(1655)

''

In [64]:
print(len(test_sentences) , len(test_sentences_hindi))

5783 5783


In [65]:
test_sentences.pop(258)
test_sentences_hindi.pop(258)
print(len(test_sentences) , len(test_sentences_hindi))

5782 5782


In [66]:
test_tensor = [[eng_lang_tokenizer.word_index[s] for s in en.split(' ')] for en in test_sentences]
# print(max_length(test_tensor))
#pad
test_sequences = pad_sequences(test_tensor, maxlen=max_input_len, padding='pre')
print(max_input_len)
# encoder_input_sequences = pad_sequences(input_tensor, maxlen=max_length_inp,padding='pre')

24


In [67]:
# i = np.random.choice(len(test_sentences))
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_bleu
count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    hindi_test = test_sentences_hindi[i : i+1]
    print('-')
    print('Input:', test_sentences[i])
    print('Response:', translation)

    print('Original Hindi Sentence:' , hindi_test[0])
    print(type(hindi_test[0]))

    


-
Input: pizza is in the foreground
Response: अग्रभूमि में पिज्जा है
Original Hindi Sentence: पिज़्ज़ा अग्रभूमि में है
<class 'str'>
-
Input: the cake on the table
Response: मेज पर केक
Original Hindi Sentence: मेज पर केक है
<class 'str'>
-
Input: red train on the tracks
Response: पटरियों पर लाल ट्रेन
Original Hindi Sentence: पटरियों पर लाल ट्रेन
<class 'str'>
-
Input: small table lamp
Response: छोटी टेबल लैंप
Original Hindi Sentence: छोटा दीप
<class 'str'>
-
Input: this is the sky
Response: यह आकाश है
Original Hindi Sentence: यह आकाश है
<class 'str'>
-
Input: a yellow plastic baseball bat
Response: एक पीला प्लास्टिक बेसबॉल बैट
Original Hindi Sentence: एक पीला प्लास्टिक बेसेबॉल
<class 'str'>
-
Input: a reflection of a mountain
Response: एक पहाड़ का प्रतिबिंब
Original Hindi Sentence: एक पहाड़ का प्रतिबिंब
<class 'str'>
-
Input: book on the bed
Response: बिस्तर पर किताब
Original Hindi Sentence: बिस्तर पर किताब
<class 'str'>
-
Input: birds in the sky
Response: आकाश में पक्षी
Original Hindi

-
Input: a wall on the side of a building
Response: किसी भवन के किनारे की एक दीवार
Original Hindi Sentence: किसी भवन के किनारे की एक दीवार
<class 'str'>
-
Input: two people on motorcycles
Response: मोटरसाइकिल पर दो लोग
Original Hindi Sentence: मोटरसाइकिल पर दो लोग
<class 'str'>
-
Input: a cat wearing a hat
Response: टोपी पहने एक बिल्ली
Original Hindi Sentence: टोपी पहने बिल्ली
<class 'str'>
-
Input: orange pail hanging from wall
Response: दीवार से लटका नारंगी लटका हुआ
Original Hindi Sentence: नारंगी की दीवार से लटकी हुई
<class 'str'>
-
Input: asian man
Response: आदमी
Original Hindi Sentence: एशियाई आदमी
<class 'str'>
-
Input: man is holding a bat
Response: आदमी एक बल्ला पकड़े हुए है
Original Hindi Sentence: मनुष्य एक बल्ला पकडा है |
<class 'str'>
-
Input: a wall on the side of a building
Response: किसी भवन के किनारे की एक दीवार
Original Hindi Sentence: किसी भवन के किनारे की एक दीवार
<class 'str'>
-
Input: large building in distance
Response: भवन में बड़ी भवन
Original Hindi Sentence: दू

In [68]:
# i = np.random.choice(len(test_sentences))
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_bleu
count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    hindi_test = test_sentences_hindi[i : i+1]
#     print('-')
#     print('Input:', test_sentences[i])
#     print('Response:', translation)

#     print('Original Hindi Sentence:' , hindi_test[0])
#     print(type(hindi_test[0]))
    
#     गिराफ़ की पूंछ होती है
#     reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
#     candidate = ['this', 'is', 'a', 'test']
#     score = sentence_bleu(reference, candidate)
#     print(score)
    
    hindi_ref = hindi_test[0]
    hindi_ref = hindi_ref.split()
#     print(hindi_ref)
    candidate = translation.split()
#     print(candidate)
    score = sentence_bleu([hindi_ref], candidate, weights=(1, 0, 0, 0) )

#     print(score)
    count = count + 1
    bleu_score = bleu_score + score
    
avg_bleu_score = bleu_score/count
print(count)
print("the average bleu score is ", avg_bleu_score)
    

C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram

100
the average bleu score is  0.6432116867615274


In [69]:

from nltk.translate.bleu_score import sentence_bleu

count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    hindi_test = test_sentences_hindi[i : i+1]

    
    hindi_ref = hindi_test[0]
    hindi_ref = hindi_ref.split()

    candidate = translation.split()

    score = sentence_bleu([hindi_ref], candidate, weights=(0, 1, 0, 0) )


    count = count + 1
    bleu_score = bleu_score + score
    
avg_bleu_score = bleu_score/count
print(count)
print("the average bleu score is ", avg_bleu_score)

100
the average bleu score is  0.41114322647569757


In [70]:
# i = np.random.choice(len(test_sentences))
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_bleu
count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    hindi_test = test_sentences_hindi[i : i+1]

    
    hindi_ref = hindi_test[0]
    hindi_ref = hindi_ref.split()

    candidate = translation.split()

    score = sentence_bleu([hindi_ref], candidate, weights=(0.25, 0.25, 0.25, 0.25) )


    count = count + 1
    bleu_score = bleu_score + score
    
avg_bleu_score = bleu_score/count
print(count)
print("the average bleu score is ", avg_bleu_score)

100
the average bleu score is  0.12505454605290592
